In [17]:
import os
import sys
import math
import wandb
import torch
import logging

import numpy as np
import pandas as pd
import torch.nn as nn
import lightning.pytorch as pl

from torchmetrics import MeanAbsoluteError
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import Callback, LearningRateMonitor, ModelCheckpoint

In [18]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# FCNN

In [19]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [20]:
#params
project_name = "MSU_interpol"

logger_path = './wandb_local_logs'
data_path = '../data/clasdb_pi_plus_n.txt'

scale_data = False
test_size = 0.1
batch_size = 32
net_architecture = [5, 60, 80, 20, 1]
lr = 0.00001
activation_function = nn.ReLU()
loss_func = RMSELoss()
optim_func = torch.optim.Adam
max_epochs = 2000
min_delta = 0.0001
patience = 20

hyperparams_dict = {
    'scale_data': scale_data,
    'test_size': test_size,
    'batch_size': batch_size,
    'lr': lr,
    'net_architecture': net_architecture,
    'activation_function': activation_function,
    'loss_func': loss_func,
    'optim_func': optim_func,
    'max_epochs': max_epochs,
    'min_delta': min_delta,
    'patience': patience
}

In [27]:
class InterpolDataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.len = len(labels)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return self.len

class InterpolDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.df = None
    
    def setup(self, stage):
        # data reading and preprocessing
        df = pd.read_csv(data_path, delimiter='\t', header=None)
        df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']

        df.loc[8314:65671, 'Ebeam'] = 5.754 # peculiarity of this dataset.
        df['phi'] = df.phi.apply(lambda x: math.radians(x))
        df = df.drop('id', axis=1)
        df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]
        self.df = df
        
        #train test split
        feature_data = df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']]
        label_data = df['dsigma_dOmega']
        
        if scale_data:
            scaler_feature = StandardScaler()
            scaler_target = StandardScaler()
            feature_data = scaler_feature.fit_transform(feature_data)
            label_data = scaler_target.fit_transform(label_data.values.reshape(-1,1))
        else:
            pass
        
        train_feature_data, val_feature_data, train_label_data, val_label_data = train_test_split(feature_data, label_data, 
                                                                                                  test_size=test_size, random_state=1438)
        
        
        self.train_dataset = InterpolDataSet(torch.tensor(train_feature_data.values, dtype=torch.float32), 
                                             torch.tensor(train_label_data.values, dtype=torch.float32))
        
        self.val_dataset = InterpolDataSet(torch.tensor(val_feature_data.values, dtype=torch.float32), 
                                            torch.tensor(val_label_data.values, dtype=torch.float32))
    def train_dataloader(self):
        return DataLoader(dataset = self.train_dataset, batch_size = batch_size, shuffle = False, num_workers=0)

    def val_dataloader(self):
        return DataLoader(dataset = self.val_dataset, batch_size = batch_size, shuffle = False, num_workers=0)

class InterpolRegressor(pl.LightningModule):
    def __init__(self, hyperparams):
        super(InterpolRegressor, self).__init__()

        self.train_loss, self.train_mae, self.val_loss, self.val_mae = 0,0,0,0
        self.hyperparams = hyperparams
        self.save_hyperparameters(self.hyperparams)

        self.mae = MeanAbsoluteError()
        self.loss_func = self.hyperparams.get('loss_func')

        self.optim = self.hyperparams.get('optim_func')

        self.net_architecture = self.hyperparams.get('net_architecture')
        self.activation_function = self.hyperparams.get('activation_function')

        self.net = nn.Sequential()
        for i in range(1,len(self.net_architecture)):
            self.net.append(nn.Linear(self.net_architecture[i-1], self.net_architecture[i]))
            if i!=len(self.net_architecture)-1:
                self.net.append(self.activation_function)
            else:
                pass

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)

        loss = self.loss_func
        self.train_loss = loss(y_hat.reshape(-1), y)

        self.train_mae = self.mae(y_hat.reshape(-1), y)

        self.log('train_loss', self.train_loss, batch_size=self.hyperparams['batch_size'],
                 on_step=False, on_epoch=True, prog_bar=True, sync_dist=True, logger=True)
        self.log('train_mae', self.train_mae, batch_size=self.hyperparams['batch_size'],
                 on_step=False, on_epoch=True, prog_bar=True, sync_dist=True, logger=True)

        return self.train_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)

        loss = self.loss_func
        self.val_loss = loss(y_hat.reshape(-1), y)
        self.val_mae = self.mae(y_hat.reshape(-1), y)

        self.log('val_loss', self.val_loss, batch_size=self.hyperparams['batch_size'],
                 on_step=False, on_epoch=True, prog_bar=True, sync_dist=True, logger=True)
        self.log('val_mae', self.val_mae, batch_size=self.hyperparams['batch_size'],
                 on_step=False, on_epoch=True, prog_bar=True, sync_dist=True, logger=True)

        return self.val_loss

    def configure_callbacks(self):
        early_stop_callback = EarlyStopping(monitor="val_loss", mode="min",
                                            min_delta=self.hyperparams.get('min_delta'),
                                            patience=self.hyperparams.get('patience'),
                                            verbose=True)

        checkpoint_callback = ModelCheckpoint(save_top_k=3,
                                              monitor="val_loss",
                                              mode="min",
                                              dirpath=f"{logger_full_path}/checkpoints",
                                              filename="{exp_name}{val_loss:.5f}-{epoch:02d}")
        print_callback = PrintCallbacks()

        return [early_stop_callback, checkpoint_callback, print_callback]

    def configure_optimizers(self):
        optimizer = self.optim(self.parameters(), lr=self.hyperparams.get('lr'))
        return optimizer

In [22]:
class PrintCallbacks(Callback):
    def on_train_start(self, trainer, pl_module):
        logging.debug("Training is starting")

    def on_train_end(self, trainer, pl_module):
        logging.debug("Training is ending")

    def on_train_epoch_end(self, trainer, pl_module):
        logging.debug(f"epoch: {pl_module.current_epoch}; train_loss: {pl_module.train_loss}; train_mae: {pl_module.train_mae}")

    def on_validation_epoch_end(self, trainer, pl_module):
        logging.debug(f"epoch: {pl_module.current_epoch}; val_loss: {pl_module.val_loss}; val_mae: {pl_module.val_mae}")

In [31]:
wandb_logger = WandbLogger(project=project_name,
                           save_dir=logger_path)
exp_name = wandb_logger.experiment.name

logger_full_path = os.path.join(logger_path, project_name, exp_name)

os.makedirs(logger_full_path, exist_ok=True)
logging.basicConfig(filename=os.path.join(logger_full_path, 'logs.log'), encoding='utf-8', level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    dirpath=os.path.join(logger_full_path, 'checkpoints'),
    filename="{exp_name}{val_loss:.5f}-{epoch:02d}",
)

early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", min_delta=min_delta, patience=patience, verbose=True)

data_module = InterpolDataModule()
model = InterpolRegressor(hyperparams=hyperparams_dict)

trainer = pl.Trainer(max_epochs=max_epochs,
                     accelerator='cpu',
                     callbacks=[PrintCallbacks(), early_stop_callback, checkpoint_callback],
                     logger=wandb_logger,
                     enable_progress_bar=False)
trainer.fit(model, data_module)

/Users/andrey.golda/Library/Caches/pypoetry/virtualenvs/msu-interpol--lw2ADYE-py3.11/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'activation_function' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['activation_function'])`.
/Users/andrey.golda/Library/Caches/pypoetry/virtualenvs/msu-interpol--lw2ADYE-py3.11/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'loss_func' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_func'])`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelC

In [25]:
wandb.finish()

epoch,▁▁
train_loss,▁
train_mae,▁
trainer/global_step,▁▁
val_loss,▁
val_mae,▁
epoch,0
train_loss,2.00194
train_mae,0.86895
trainer/global_step,2627
val_loss,1.79344


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1002)'))': /api/4504800232407040/envelope/
Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1002)'))': /api/4504800232407040/envelope/
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1002)'))': /api/4504800232407040/envelope/
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection b